<a href="https://colab.research.google.com/github/wizkers007/fooddetection/blob/main/Food_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report
from google.colab import drive
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.mixed_precision import set_global_policy
#set_global_policy('mixed_float16')  # Enables mixed precision training

drive.mount('/content/drive')

image_dir = Path('/content/drive/MyDrive/Images3')
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    sample_size = min(2000, len(category_slice))  # Ensure sample size is no larger than available items
    category_samples.append(category_slice.sample(sample_size, random_state=1))

image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
image_df['Label'].value_counts()


In [ ]:
num_classes = image_df['Label'].nunique()
print(num_classes)  # This should print 10

In [ ]:
train_df, test_df = train_test_split(image_df, train_size=0.8, shuffle=True, random_state=42)
#was train_size=0.7 earlier

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
validation_split=0.25
)
#was validation_split= 0.2 earlier
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
unique_classes = image_df['Label'].unique().tolist()
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=128,
    shuffle=True,
    seed=42,
    subset='training',
    classes=unique_classes  # Ensure consistent class order
)
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=128,
    shuffle=True,
    seed=42,
    subset='validation',
    classes=unique_classes  # Ensure consistent class order
)
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=128,
    shuffle=False,
    classes=unique_classes  # Ensure consistent class order
)


In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

In [ ]:
# Freeze the pretrained model
pretrained_model.trainable = False

# Define the new model with 5 output classes
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)  # Changed to 5 classes

model = tf.keras.Model(inputs, outputs)

# Print the model summary
print(model.summary())


In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Ensure labels are one-hot encoded
    metrics=['accuracy']
)

# Train the model using train_images and validate with val_images
history = model.fit(
    train_images,  # No need for train_labels, it's included in train_images
    validation_data=val_images,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)


In [ ]:
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

